<a href="https://colab.research.google.com/github/RReed637/Suicide-Risk/blob/main/Notebook/SuicideWatchRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyprind
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install plotly
!pip install plotly.io
!pip install plotly.express
!pip install tensorflow
!pip install keras
!pip install sklearn
!pip install pandas
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


ERROR: Could not find a version that satisfies the requirement plotly.io (from versions: none)
ERROR: No matching distribution found for plotly.io
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0

In [3]:
import pandas as pd
import pyprind
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.io as pio


## RNN


#### Creating datasets

In [4]:

# Step 3: Define Custom Dataset Class
class CustomDataset():
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx, 0]  # Assuming text is in the first column
        label = self.dataframe.iloc[idx, 1]  # Assuming label is in the second column
        return text, label


In [5]:
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

train_on_gpu = True

# Load the CSV file into a DataFrame, specifying the delimiter
df = pd.read_csv(r'/content/cleaned_data.csv', delimiter=',', header=None)  # Assuming no header

df = df.reset_index(drop=True)
# Rename the class column to "label"
df = df.rename(columns={0: 'text', 1: 'label'})
df = df.drop(0)
print(df.head())

# Split the data into train, validation, and test sets
train_valid_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, valid_df = train_test_split(train_valid_df, test_size=0.25, random_state=42)  # Splitting 80% for train and 20% for validation

# Process tokens to remove extra characters (such as commas, single quotes, and square brackets)
def preprocess_tokens(sentence):
    return [word.strip(",'[]") for word in sentence.split()]

# Create a counter to count word frequencies
word_counter = Counter()

# Count word frequencies from the training data
for sentence in train_df['text']:
    word_counter.update(preprocess_tokens(sentence))

# Create a vocabulary ordered by word frequency
vocab_ordered = {'<pad>': 0, '<unk>': 1}
word_index = len(vocab_ordered)
for word, count in word_counter.most_common():
    vocab_ordered[word] = word_index
    word_index += 1

# Optionally, you can save the vocab to a file for later use
with open('vocab_ordered.txt', 'w') as f:
    for word, index in vocab_ordered.items():
        f.write(f"{word}: {index}\n")

# Now you have train_df, valid_df, test_df DataFrames with the label column renamed



                                                text label
1  ['wife', 'threaten', 'suicide', 'recently', 'l...     1
2  ['weird', 'get', 'affect', 'come', 'someone', ...     0
3  ['finally', 'almost', 'never', 'hear', 'bad', ...     0
4    ['need', 'help', 'just', 'help', 'cry', 'hard']     1
5  ['lose', 'hello', 'name', 'struggle', 'year', ...     1


# THIS IS THE PROBLEM!!!

# TASKS FOR Today:
### Build 3 RNN Models using these three types of recurrent layers
#### RNN
#### LSTM
#### GRU


In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn

# Define text and label pipelines
text_pipeline = lambda x: [vocab_ordered[word] if word in vocab_ordered else vocab_ordered['<unk>'] for word in preprocess_tokens(x)]
label_pipeline = lambda x: torch.tensor([int(x), 1 - int(x)], dtype=torch.float32)


def collate_batch(batch):
    text_list, label_list = [], []
    for text, label in batch:
        label_list.append(label_pipeline(label))
        processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
        text_list.append(processed_text)

    label_list = torch.stack(label_list, dim = 0)

    padded_text_list = nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=0)

    # Create a dictionary containing the processed text, labels, and lengths
    batch_dict = {
        'text': padded_text_list,
        'label': label_list
    }

    return batch_dict

# Assuming you have already defined your training, validation, and test DataFrames as train_df, valid_df, and test_df respectively

# Instantiate CustomDataset objects
train_dataset = CustomDataset(train_df)
valid_dataset = CustomDataset(valid_df)
test_dataset = CustomDataset(test_df)

# Define batch size
batch_size = 15

# Create DataLoader objects for training, validation, and test sets
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
valid_dl = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)




In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split, KFold








class LSTMModel(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim


        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)


        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                           batch_first=True)  # LSTM layer
        self.dropout = nn.Dropout(dropout)
        self.global_max_pooling = nn.AdaptiveMaxPool1d(1)  # Global max pooling

        self.fc = nn.Linear(hidden_dim, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        x = x.long()
        embedded = self.embedding(x)
        lstm_output, hidden = self.lstm(embedded)
        lstm_output = self.dropout(lstm_output)
        out = self.fc(lstm_output[:, -1])  # Apply linear layer to last output
        output = torch.sigmoid(out)  # Apply sigmoid activation
        return output, hidden



    def init_hidden(self, batch_size):
      '''Initialize Hidden State'''
      weight = next(self.parameters()).data

      hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

      if torch.cuda.is_available():
          hidden = (hidden[0].cuda(), hidden[1].cuda())

      return hidden






In [40]:

from torch.optim.lr_scheduler import ReduceLROnPlateau
# Defining Hyperparameters
vocab_size = len(vocab_ordered)+1
output_size = 2
embedding_dim = [16, 32, 64]
hidden_dim = [32, 64, 128]
n_layers = [1, 2, 3, 4, 5]
dropout = [0.1, 0.2, 0.3, 0.4, 0.5]
lr = [0.001, 0.0001, 0.00001]
batch_size = [16, 32]

#Performing K-fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

best_hyperparameters = {}
best_val_acc = 0.0


for embedding_dim in embedding_dim:
  for hidden_dim in hidden_dim:
    for n_layers in n_layers:
      for dropout in dropout:
        for lr in lr:
          for batch_size in batch_size:
            val_accuracies = []
            for train_index, val_index in kf.split(train_valid_df):
              train_df = CustomDataset(train_valid_df.iloc[train_index])
              val_df = CustomDataset(train_valid_df.iloc[val_index])

              train_dl = DataLoader(train_df, batch_size=batch_size, shuffle=True)
              val_dl = DataLoader(val_df, batch_size=batch_size, shuffle=False)

              net = LSTMModel(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
              #Loss Function and Optimizer
              criterion = nn.BCELoss()
              optimizer = optim.Adam(net.parameters(), lr=lr)

              precision_scores_train, recall_scores_train, f1_scores_train, roc_auc_scores_train, precision_scores_val, recall_scores_val, f1_scores_val, roc_auc_scores_val = train()



            avg_val_acc = sum(val_accuracies) / len(val_accuracies)
            if avg_val_acc > best_val_acc:
              best_val_acc = avg_val_acc
              best_hyperparameters = {
                  'vocab_size': vocab_size,
                  'output_size': output_size,
                  'embedding_dim': embedding_dim,
                  'hidden_dim': hidden_dim,
                  'n_layers': n_layers,
                  'dropout': dropout,
                  'lr': lr,
                  'batch_size': batch_size
              }

print("Best Hyperparameters:", best_hyperparameters)
print("Best Validation Accuracy:", best_val_acc)



KeyError: 17583

## RNN


#### Creating datasets

In [13]:
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    # d_model must be divisable by the number of heads
    assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

    #Initialize Dimensions
    self.d_model = d_model #Model's Dimension
    self.num_heads = num_heads # Number of attention heads
    self.d_k = d_model // num_heads # Dimension of each head's key, query, and value

    # Linear Layers for transforming inputs

    self.W_q = nn.Linear(d_model, d_model)

class TransformerBlock(nn.Module):
  def __init__(self, embed_dim, num_heads, ff_dim, rate):
    super(TransformerBlock, self).__init__()
    self.att =



SyntaxError: invalid syntax (<ipython-input-13-f543cee718e4>, line 22)

In [ ]:

def train_model(dataloader):
    total_acc, total_loss = 0.0, 0.0
    for i, batch in enumerate(dataloader):
        print(f"Batch Index: {i}")
        text_batch = batch['text']
        label_batch = batch['label']
        lengths = batch['length']

        num_classes = 2  # Number of classes in your classification task
        label_batch_one_hot = torch.zeros(label_batch.size(0), num_classes, dtype=torch.float32)  # Initialize with float dtype
        label_batch_one_hot.scatter_(1, label_batch.unsqueeze(1).long(), 1)  # Convert label_batch to int64 using .long()

        h_prev = np.zeros((hidden_size, text_batch.shape[1]))  # Initialize hidden state
        print("Initial h_prev:", h_prev)
        loss = rnn_model.train_step(text_batch, label_batch, h_prev)
        print("Updated h_prev:", h_prev)

        total_loss += loss * text_batch.shape[1]

        pred = rnn_model.forward(text_batch, h_prev)[0]
        total_acc += ((pred >= 0.5).astype(float) == label_batch).astype(float).sum().item()

    return total_acc / len(dataloader.dataset), total_loss / len(dataloader.dataset)

In [ ]:
def evaluate(dataloader):
    total_acc, total_loss = 0.0, 0.0
    for i, batch in enumerate(dataloader):
        print(f"Batch Index: {i}")
        text_batch = batch['text']
        label_batch = batch['label']
        lengths = batch['length']

        h_prev = np.zeros((hidden_size, text_batch.shape[1]))  # Initialize hidden state
     # Initialize hidden state

        loss = rnn_model.train_step(text_batch, label_batch, h_prev)

        total_loss += loss * text_batch.shape[0]

        pred = rnn_model.forward(text_batch, h_prev)[0]
        total_acc += ((pred >= 0.5).astype(float) == label_batch).astype(float).sum().item()

    return total_acc / len(dataloader.dataset), total_loss / len(dataloader.dataset)


In [ ]:

torch.manual_seed(1)
for epoch in range(num_epochs):
    acc_train, loss_train = train_model(train_dl)
